<a href="https://colab.research.google.com/github/OscarFloresP/MachineLearning-TP/blob/main/ML_TP_Grupo_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning - TA y TB 

+ Alessandro Carhuancho - u201913933
+ Oscar Flores - u201716498

### Archivo de ejemplo

In [1]:
%%writefile sample.off
OFF
8 6 0
-0.500000 -0.500000 0.500000
0.500000 -0.500000 0.500000
-0.500000 0.500000 0.500000
0.500000 0.500000 0.500000
-0.500000 0.500000 -0.500000
0.500000 0.500000 -0.500000
-0.500000 -0.500000 -0.500000
0.500000 -0.500000 -0.500000
4 0 1 3 2
4 2 3 5 4
4 4 5 7 6
4 6 7 1 0
4 1 7 5 3
4 6 0 2 4

Writing sample.off


### Lectura y escritura de archivo STL binario

Versión de código extraída y modificada de: \
> Christoph, S. (2021). Tweaker - Auto Rotation Module for FDM 3D Printing (Version v3.9.1) [Computer software]. https://doi.org/10.13140/RG.2.2.27593.36966


In [2]:
import sys
import os
import struct
import time
import traceback
import numpy as np

In [3]:
class StlbFileHandler:
    def __init__(self):
        pass

    @staticmethod
    def load_binary_stl(filename:str):
        """Load the content of a binary STL file."""
        fh = open(filename,"rb")
        fh.seek(5, os.SEEK_SET)
        # Skip the header
        fh.read(80 - 5)
        face_count = struct.unpack('<I', fh.read(4))[0]
        objects = dict()
        objects[0] = {"mesh": list(), "name": "binary file"}
        for idx in range(0, face_count):
            data = struct.unpack("<ffffffffffffH", fh.read(50))
            objects[0]["mesh"].append([data[3], data[4], data[5]])
            objects[0]["mesh"].append([data[6], data[7], data[8]])
            objects[0]["mesh"].append([data[9], data[10], data[11]])
        return objects

    @classmethod
    def write_mesh(cls, objects, outputfile):
        header = "Tweaked on {}".format(time.strftime("%a %d %b %Y %H:%M:%S")
                                        ).encode().ljust(79, b" ") + b"\n"
        for part, content in objects.items():
            mesh = objects[part]["mesh"]
            partlength = int(len(mesh) / 3)
            mesh = cls.rotate_bin_stl(mesh)

            if len(objects.keys()) == 1:
                outname = outputfile
            else:
                outname = ".".join(outputfile.split(".")[:-1]) + "_{}.stl".format(part)
            length = struct.pack("<I", partlength)
            with open(outname, 'wb') as outfile:
                outfile.write(bytearray(header + length + b"".join(mesh)))

    @classmethod
    def rotate_bin_stl(cls, content):
        mesh = np.array(content, dtype=np.float64)

        # prefix area vector, if not already done (e.g. in STL format)
        if len(mesh[0]) == 3:
            row_number = int(len(content) / 3)
            mesh = mesh.reshape(row_number, 3, 3)

        v0 = mesh[:, 0, :]
        v1 = mesh[:, 1, :]
        v2 = mesh[:, 2, :]
        normals = np.cross(np.subtract(v1, v0), np.subtract(v2, v0)
                           ).reshape(int(len(mesh)), 1, 3)
        mesh = np.hstack((normals, mesh))
        mesh = list(map(cls.write_bin_facett, mesh))

        return mesh

    @staticmethod
    def write_bin_facett(facett):
        tweaked = struct.pack("<fff", facett[0][0], facett[0][1], facett[0][2])
        tweaked += struct.pack("<fff", facett[1][0], facett[1][1], facett[1][2])
        tweaked += struct.pack("<fff", facett[2][0], facett[2][1], facett[2][2])
        tweaked += struct.pack("<fff", facett[3][0], facett[3][1], facett[3][2])
        tweaked += struct.pack("<H", 0)

        return tweaked

### Lectura, Escritura y Conversión de archivos OFF, OBJ y STL (ASCII y binario)

Se usa el anterior módulo como liberia para los archivos binarios

In [4]:
import itertools
from enum import Enum

In [5]:
class Filetype:
    class FT(str, Enum):
        OFF = 'off'
        OBJ = 'obj'
        STL = 'stl'

    class Utils:
        @staticmethod
        def off_wrap(vertices:list, faces:list):
            return ["OFF"] + [f"{len(vertices)} {len(faces)} 0"] + vertices + faces

        @staticmethod
        def off_polygon2triangle(polygon_as:str):
            # ignore the first 2 characters as this are NVertices
            poly = [ i for i in polygon_as[2:].split()]
            return [[poly[0], b, c] for b, c in zip(poly[1:], poly[2:])]

        @staticmethod
        def stl_ascii_wrap(lines:list):
            return ["solid model"] + lines + ["endsolid model"]


In [6]:
class BaseIO:
    @staticmethod
    def read(filename:str) -> str:  
        with open(filename, 'r') as fh:
            content = fh.read()
        return content

    @staticmethod
    def writelines(filename:str, lines:list):
        with open(filename, 'w') as fh:
            for line in lines:
                fh.write(line + "\n")

In [7]:
class Parser(BaseIO, Filetype.Utils):
    def __map_methods__(self):
        self.__r__ = { 'off': self.from_off, 'obj': self.from_obj, 'stl': self.from_stl }
        self.__w__ = { 'off': self.to_off, 'obj': self.to_obj, 'stl': self.to_stl }

    # Read data and preserve faces and vertices information
    def __init__(self, filename:str, ft:Filetype.FT, stlb=False):
        self.__map_methods__()
        self.ft, self.filename = ft, filename
        if stlb and ft.value == "stl":
            self.from_stl_bin()
        else:
            self.__r__[ft.value](self.read(filename).splitlines())

    def write(self, ft:Filetype.FT, new_filename="", stlb=False):
        if stlb and ft.value == "stl":
            self.to_stl_bin(new_filename)
        else:
            self.__w__[ft.value](new_filename)

    @staticmethod
    def new_filename(filename:str, new_filename:str, ft:Filetype.FT):
        if new_filename == "":
            extension, name = ft.value, filename.split('.')
            if len(name) > 1:
                return ".".join(name[:-1] + [extension])
            return ".".join(name + [extension])
        return new_filename

    def from_off(self, lines: list):
        numVertices = int(lines[1][0])
        # get vertices and faces
        self.vertices = [line.split() for line in lines[2: numVertices + 2]]
        # get faces, but parse polygons into triangles
        triangles = [self.off_polygon2triangle(line) for line in lines[numVertices + 2:]]
        self.triangles = list( itertools.chain(*triangles) )

    def from_obj(self, lines: list):
        self.vertices = [ line.split()[1:] for line in lines if line[0] == "v" ]
        self.triangles = [ line.split()[1:]  for line in lines if line[0] == "f" ]

    def from_stl(self, lines: list):
        temp = [ line.split()[1:] for line in lines if line.split()[0] == "vertex" ]
        self.vertices = [ list(xyz) for xyz in set([ tuple(e) for e in temp]) ]
        mapv = { tuple(xyz): str(i) for i, xyz in enumerate(self.vertices) }
        self.triangles = [ [mapv[tuple(temp[k + i])] for i in range(3)] for k in range(0, len(temp), 3) ]      

    def from_stl_bin(self):
        temp = StlbFileHandler.load_binary_stl(self.filename)[0]["mesh"]
        self.vertices = [ [str(e) for e in xyz] for xyz in set([ tuple(e) for e in temp]) ]
        # print(self.vertices)
        mapv = { tuple(xyz): str(i) for i, xyz in enumerate(self.vertices) }
        # print(mapv)
        self.triangles = [ [mapv[tuple([ str(e) for e in temp[k + i]])] for i in range(3)] for k in range(0, len(temp), 3) ] 

    def to_off(self, new_filename:str):
        new_filename = self.new_filename(self.filename, new_filename, Filetype.FT.OFF)
        vertices = [ (" ".join([xyz for xyz in vertex])) for vertex in self.vertices ]
        faces = [ ("3 "+" ".join([v_id for v_id in triangle])) for triangle in self.triangles ]
        self.writelines(new_filename, self.off_wrap(vertices, faces))

    def to_obj(self, new_filename:str):
        new_filename = self.new_filename(self.filename, new_filename, Filetype.FT.OBJ)
        vertices = [ ("v " + " ".join([xyz for xyz in vertex])) for vertex in self.vertices ]
        faces = [ ("f "+" ".join([v_id for v_id in triangle])) for triangle in self.triangles ]
        self.writelines(new_filename, vertices + faces)
        
    def to_stl(self, new_filename:str):
        new_filename = self.new_filename(self.filename, new_filename, Filetype.FT.STL)

        blocks = [f"""facet normal 0.0 0.0 0.0
    outer loop
        vertex {" ".join([xyz for xyz in self.vertices[ int(triangle[0]) ] ])}
        vertex {" ".join([xyz for xyz in self.vertices[ int(triangle[1]) ] ])}
        vertex {" ".join([xyz for xyz in self.vertices[ int(triangle[2]) ] ])}
    endloop
endfacet""" for triangle in self.triangles]
        self.writelines(new_filename, self.stl_ascii_wrap(blocks))

    def to_stl_bin(self, new_filename:str):
        new_filename = self.new_filename(self.filename, new_filename, Filetype.FT.STL)

        mesh = [([ float(xyz) for xyz in self.vertices[ int(triangle[0]) ] ], 
                 [ float(xyz) for xyz in self.vertices[ int(triangle[1]) ] ],
                 [ float(xyz) for xyz in self.vertices[ int(triangle[2]) ] ]) for triangle in self.triangles ]     

        objects = dict()
        objects[0] = {"mesh": list( itertools.chain(*mesh) ), "name": "binary file"}
        StlbFileHandler.write_mesh(objects, new_filename)

In [8]:
x = Parser("sample.off", Filetype.FT.OFF)

x.write(Filetype.FT.OBJ)
x.write(Filetype.FT.STL)
x.write(Filetype.FT.STL, "sample_bin.stl", stlb=True)

In [9]:
# x = Parser("cube.stl", Filetype.FT.STL, stlb=True)

# x.write(Filetype.FT.STL, "cube1.stl", stlb=True)